# Census Data Tools

MORPC works regularly with census data, including but not limited to ACS 5 and 1-year, Decennial Census, PEP, and geographies. The following module is useful for gathering and organizing census data for processes in various workflow. Those workflows are linked when appropriate. 

In [ ]:
import morpc

## API functions and variables

api_get() is a low-level wrapper for Census API requests that returns the results as a pandas dataframe. If necessary, it splits the request into several smaller requests to bypass the 50-variable limit imposed by the API.  

The resulting dataframe is indexed by GEOID (regardless of whether it was requested) and omits other fields that are not requested but which are returned automatically with each API request (e.g. "state", "county") 

In [ ]:
url = 'https://api.census.gov/data/2022/acs/acs1'
params = {
    "get": "GEO_ID,NAME,B01001_001E",
    "for": "county:049,041",
    "in": "state:39"
}

In [ ]:
api = morpc.census.api_get(url, params)

In [ ]:
api

## American Community Survey (ACS) Data Class

When using ACS data, generally we will be digesting data produded using the [morpc-censusacs-fetch](https://github.com/morpc/morpc-censusacs-fetch) workflow. The data that is produced from that script is by default saved in its output_data folders ./morpc-censusacs-fetch/output_data/

The Census ACS Fetch script leverages the `acs_data` class form `morpc.census`


### Create an initial object which represents a variable in the ACS data api.

The class takes 3 arguments:

1. variable group number
2. the year
3. the type of survey (1 or 5 year estimates)

In [ ]:
import morpc

In [ ]:
acs = morpc.census.acs_data('B02001', '2023', '5')

The initial call creates queries the Census for the variable definitions and returns a dictionary of the available variables in the group. see `acs.VARS`

In [ ]:
acs.VARS

### Query the API for the deisred variables and geography

The `.query()` method queries the API and caches the data in memory under `acs.DATA`. At the same time it creates a frictionless schema that corrosponds with the data. 

#### scope:
These are pre-defined sumlevels and scopes for commonly queried geographies. see `morpc.census.SCOPES`.

In [ ]:
morpc.census.SCOPES

In [ ]:
morpc.census.ACS_VAR_GROUPS['B02001']['dimensions']

In [ ]:
acs = acs.query(scope='region15-counties')

In [ ]:
data = acs.DATA
data.head()

### For custom queries, use for and in parameters to pass to api query. 

#### for_param:
(optional) The geographies for which to call the the query "state:*" represents all states. "state:39" represent Ohio.

#### in_param:
(optional) A filter for the for parameter. In combinations this allows you do call for small geograhpies inside larger ones. 

> Examples: for_param="county:\*", in_param="state:39" would get all counties in Ohio.
> for_param="tract:\*", in_param='state:39,county:041,049' gets all census tracts in Delaware and Franklin Counties.

### Filter the variables using the get parameter

#### get_param:
(Optional) If you want to return a subset of variables, they can be passed here as a list.

### Dimension Tables

When the query is called the class makes table with the dimensions included that can be used to get summaries of the data. 

This can be used to get quick queries for summaries. 

In [1]:
import morpc

In [2]:
acs = morpc.census.acs_data('B01001', '2023', '5').query(scope='region15-counties')

morpc-acs5-2023-region15-counties-b01001 schema is valid
Total variables requested: 100
Starting request #1. 100 variables remain.
Starting request #2. 81 variables remain.
Starting request #3. 63 variables remain.
Starting request #4. 45 variables remain.
Starting request #5. 27 variables remain.
Starting request #6. 9 variables remain.


In [3]:
acs.DIM_TABLE.LONG

,GEO_ID,NAME,VARIABLE,VALUE,VAR_TYPE,TOTAL,Sex,Age,REFERENCE_YEAR
0,0500000US39041,"Delaware County, Ohio",B01001_001E,221160,Estimate,Total,Total,Total,2023
1,0500000US39045,"Fairfield County, Ohio",B01001_001E,161289,Estimate,Total,Total,Total,2023
2,0500000US39047,"Fayette County, Ohio",B01001_001E,28880,Estimate,Total,Total,Total,2023
3,0500000US39049,"Franklin County, Ohio",B01001_001E,1321635,Estimate,Total,Total,Total,2023
4,0500000US39073,"Hocking County, Ohio",B01001_001E,27938,Estimate,Total,Total,Total,2023
...,...,...,...,...,...,...,...,...,...
1465,0500000US39117,"Morrow County, Ohio",B01001_049M,107,MOE,Total,Female,85 years and over,2023
1466,0500000US39127,"Perry County, Ohio",B01001_049M,71,MOE,Total,Female,85 years and over,2023
1467,0500000US39129,"Pickaway County, Ohio",B01001_049M,112,MOE,Total,Female,85 years and over,2023
1468,0500000US39141,"Ross County, Ohio",B01001_049M,167,MOE,Total,Female,85 years and over,2023


In [30]:
acs.DIM_TABLE.WIDE

GEO_ID                          0500000US39041         0500000US39045  \
NAME                     Delaware County, Ohio Fairfield County, Ohio   
REFERENCE_YEAR                            2023                   2023   
Sex    Age                                                              
Total  Total                            221160                 161289   
Male   Total                            110668                  80584   
       Under 5 years                      6456                   4690   
       5 to 9 years                       7560                   5634   
       10 to 14 years                     9200                   5865   
       15 to 17 years                     5478                   3637   
       18 and 19 years                    3068                   1976   
       20 years                           1337                   1096   
       21 years                           1490                    857   
       22 to 24 years                     3434                   3135   
       25 to 29 years                     4556                   4917   
       30 to 34 years                     5846                   4917   
       35 to 39 years                     7477                   5557   
       40 to 44 years                     9403                   5248   
       45 to 49 years                     8681                   5266   
       50 to 54 years                     8309                   5460   
       55 to 59 years                     7351                   5779   
       60 and 61 years                    2678                   1832   
       62 to 64 years                     3255                   2844   
       65 and 66 years                    2304                   1912   
       67 to 69 years                     2872                   2251   
       70 to 74 years                     4734                   3185   
       75 to 79 years                     2642                   1901   
       80 to 84 years                     1212                   1497   
       85 years and over                  1325                   1128   
Female Total                            110492                  80705   
       Under 5 years                      6078                   4611   
       5 to 9 years                       7775                   4596   
       10 to 14 years                     8150                   6108   
       15 to 17 years                     5177                   3457   
       18 and 19 years                    2782                   1580   
       20 years                           1353                    867   
       21 years                           1370                    974   
       22 to 24 years                     3133                   2559   
       25 to 29 years                     4701                   4441   
       30 to 34 years                     6186                   5011   
       35 to 39 years                     8321                   5452   
       40 to 44 years                     8575                   5453   
       45 to 49 years                     8251                   5091   
       50 to 54 years                     7863                   5327   
       55 to 59 years                     7343                   5146   
       60 and 61 years                    2459                   2607   
       62 to 64 years                     3557                   2852   
       65 and 66 years                    2722                   1783   
       67 to 69 years                     2937                   2845   
       70 to 74 years                     4738                   3635   
       75 to 79 years                     3408                   2575   
       80 to 84 years                     1604                   1843   
       85 years and over                  2009                   1892   

GEO_ID                         0500000US39047        0500000US39049  \
NAME                     Fayette County, Ohio Franklin Coun

In [29]:
acs.DIM_TABLE.WIDE.loc(axis=0)[('Male', 'Female'),'Total'].T

,,Sex,Male,Female
,,Age,Total,Total
GEO_ID,NAME,REFERENCE_YEAR,,
0500000US39041,"Delaware County, Ohio",2023,110668,110492
0500000US39045,"Fairfield County, Ohio",2023,80584,80705
0500000US39047,"Fayette County, Ohio",2023,14271,14609
0500000US39049,"Franklin County, Ohio",2023,649003,672632
0500000US39073,"Hocking County, Ohio",2023,14022,13916
0500000US39083,"Knox County, Ohio",2023,31021,31867
0500000US39089,"Licking County, Ohio",2023,89196,91115
0500000US39091,"Logan County, Ohio",2023,23206,22934


### Save raw data (not dim table) as a frictionless resource with schema

After querying the data, save the data as a frictionless resource with reasonable descriptors. 

In [ ]:
acs.save(output_dir='./temp_data/')

In [ ]:
acs.SCHEMA

In [ ]:
acs.RESOURCE

## Georeference the data to map

In [ ]:
acs = morpc.census.acs_data('B01001', '2023', '5')
acs = acs.query(get_param=['B01001_001E'], scope='region15-tracts')

In [ ]:
acs = acs.georeference()

In [ ]:
acs.DATA.explore(column='B01001_001E')

## Below should still be functional, but hoping to implement into ACS class

#### Load the data using frictionless.load_data()

In [ ]:
data, resource, schema = morpc.frictionless.load_data('./temp_data/morpc-acs5-2023-state-B01001.resource.yaml', verbose=False)

#### Using ACS_ID_FIELDS to get the fields ids

In [ ]:
morpc.census.acs_generate_universe_table(data.set_index("GEO_ID"), "B01001_001")

#### Create a dimension table with the data and the dimension names

In [ ]:
dim_table = morpc.census.acs_generate_dimension_table(data.set_index("GEO_ID"), schema, idFields=idFields, dimensionNames=["Sex", "Age group"])

In [ ]:
dim_table.loc[dim_table['Variable type'] == 'Estimate'].head()

### Build ACS Variable Group JSON for Dimension names

In [ ]:
import requests
r = requests.get('https://api.census.gov/data/2023/acs/acs5/variables.json')
varjson = r.json()

In [ ]:
groups = {}
for variable in varjson['variables']:
    if variable not in ['for', 'in', 'ucgid', 'GEO_ID', 'AIANHH', 'AIHHTL', 'AIRES', 'ANRC']:
        group = varjson['variables'][variable]['group']
        if not group[-1].isalpha():
            if group not in groups:
                groups[group] = {}
                groups[group]['concept'] = varjson['variables'][variable]['concept'].replace('Year and Over','Year & Over').replace('Indian and Alaska','Indian & Alaska').replace('Hawaiian and Other','Hawaiian & Other')
                groups[group]['dimensions'] = ['TOTAL'] + varjson['variables'][variable]['concept'].replace(' by ',':').replace('Year and Over','Year & Over').replace('Indian and Alaska','Indian & Alaska').replace('Hawaiian and Other','Hawaiian & Other').replace(' and ',':').split(':')
                groups[group]['dimensions_verified'] = False
                variables = {}
                for variable in varjson['variables']:
                    if varjson['variables'][variable]['group'] == group:
                        variables[variable] = varjson['variables'][variable]['label']
                variables = {k: v for k, v in sorted(variables.items(), key=lambda item: item[0])}
                groups[group]['variables'] = variables

In [ ]:
groups = {k: v for k, v in sorted(groups.items(), key=lambda item: item[0])}

In [ ]:
import json
with open('../morpc/census/acs_variable_groups.json', 'w') as file:
    json.dump(groups, file, indent=3)